In [16]:
import getpass
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""
os.environ["USER_AGENT"]="Tanmay"

In [17]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace

In [18]:
llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat = ChatHuggingFace(llm=llm)

In [19]:
class RouteQuery(BaseModel):

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )


In [20]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(name="datasource", description="Given a user question choose which datasource would be most relevant for answering their question. the three datasource options are python_docs, js_docs, golang_docs"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()


In [90]:
structured_llm = chat.with_structured_output(RouteQuery)

In [96]:
structured_llm.get_output_schema()

langchain_core.output_parsers.openai_tools.PydanticToolsParserOutput

In [26]:
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_template("""
You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source.

Question: {question}

Instructions: {instructions}
""").partial(instructions=format_instructions)

router = prompt | chat | output_parser


In [27]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})


In [28]:
result

{'datasource': 'python_docs'}